In [6]:
a = {
    # "buildship": "https://github.com/eclipse/steady.git",
    # "eclips-collections": "https://github.com/eclipse/eclipse-collections.git",
    # "hawkbit": "https://github.com/eclipse/hawkbit.git",
    "jifa" : " ",
    "jkube": " ",
    # "kura": "https://github.com/eclipse/kura.git",
    # "milo": "https://github.com/eclipse/milo.git",
    # "openvsx": "https://github.com/eclipse/openvsx.git",
    # "steady": "https://github.com/eclipse/steady.git",
    # "xtext": "https://github.com/eclipse/xtext.git"

}

In [2]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 48.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 8.2 MB/s eta 0:00:00
  Created wheel for ktrain: filename=ktrain-0.41.0-py3-none-any.whl size=25316877 sha256=3367b7aedb57a87aaeef8f25486e3753f256c1419e8e518d024c2ed8b52383d6
  Stored in directory: /root/.cache/pip/wheels/31/61/66/30b156

In [3]:
! pip install https://github.com/amaiya/eli5-tf/archive/refs/heads/master.zip

     \ 6.9 MB 17.6 MB/s 0:00:00m
  Preparing metadata (setup.py) ... done


In [3]:
import ktrain
from ktrain import text
import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [4]:
def code_bert(project='buildship'):
  MODEL_NAME = 'microsoft/codebert-base'
  t = text.Transformer(MODEL_NAME, maxlen=512, class_names=['not_LA', 'is','set','get'])

  df = pd.read_csv(f"../data/{project}_final.csv")
  df = df.drop(df.index)
  print(df.shape)
  for p,u in a.items():
    if p == project:
        continue
    df1 = pd.read_csv(f'../data/{p}_final.csv')
    df = pd.concat([df,df1])
  df = shuffle(df)
  train_x = df['content'].values
  df.drop(columns=['content',"return_type","method_name","parameters","returns","comments","access_modifier"],inplace=True,axis=1)
  train_y = df.values
  test_df = pd.read_csv(f'../data/{project}_final.csv')
  test_x = test_df['content'].values
  test_df.drop(columns=['content',"return_type","method_name","parameters","returns","comments","access_modifier"],inplace=True,axis=1)
  test_y = test_df.values
  

  
  

  # # access_modifier,return_type,method_name,parameters,returns,comments,content,label
  # df.dropna(subset=['content'])
  # X = df['content'].tolist()
  # y = df['catagory'].to_list()
  
#   # df['combined'] = df['access_modifier']+' '+df['return_typed']+' '+df['method_name']+'('+df['parameters']+')'+df['content']
#   # y = df['label'].map({'positive': 1, 'negative': 0}).values
#   # Manually split your dataset into training and validation sets if not already done
#   from sklearn.model_selection import train_test_split
#   X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
  # Note: Adjust maxlen according to your data's characteristics
  trn = text.Transformer(MODEL_NAME, maxlen=512, class_names=['not_LA', 'is','set','get'])
  train_data = trn.preprocess_train(train_x, train_y)
  val_data = trn.preprocess_test(test_x, test_y)
  # Get classifier model
  model = trn.get_classifier(multilabel=True)

  # Create a Learner object
  learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=8)
      # Train the model
  learner.fit_onecycle(5e-5, 1)
      # Validate the model
  val_metrics = learner.validate(class_names=['not_LA', 'is','set','get'])
  print("Validation Metrics:", val_metrics)
  predictor = ktrain.get_predictor(learner.model, preproc=t)

  pred = predictor.predict(test_x,return_proba=True)
  print(pred)
  # prompt: add the code to plot the roc curve for my code
  data = {"get":0,"is":1,"not_LA":2,"set":3}

  # Convert list to DataFrame
  # test_df1 = pd.DataFrame(y_val, columns=['cata'])

  # fpr = dict()
  # tpr = dict()
  # roc_auc = dict()

  # for class_name, class_idx in data.items():
  #     y_true_class = (test_df1['cata'] == class_name).astype(int)

  #     y_score_class = pred[:, class_idx]
  #     fpr[class_name], tpr[class_name], _ = roc_curve(y_true_class, y_score_class)
  #     roc_auc[class_name] = auc(fpr[class_name], tpr[class_name])
  #         # Plot ROC curve for each class
  # plt.figure()
  # for class_name, class_idx in data.items():
  #     plt.plot(fpr[class_name], tpr[class_name], label=f'ROC curve (AUC = {roc_auc[class_name]:0.2f}) for {class_name}')
  # plt.plot([0, 1], [0, 1], 'k--')
  # plt.xlim([0.0, 1.0])
  # plt.ylim([0.0, 1.05])
  # plt.xlabel('False Positive Rate')
  # plt.ylabel('True Positive Rate')
  # plt.title('Receiver Operating Characteristic (ROC) Curve')
  # plt.legend(loc="lower right")
  # plt.plot()
  print(predictor.predict('public void setupRegistry() { stateBeforeInjectorCreation = GlobalRegistries.makeCopyOfGlobalState(); if (injector == null) { getInjector(); } stateAfterInjectorCreation.restoreGlobalState(); }'))
  predictor.explain('public void setupRegistry() { stateBeforeInjectorCreation = GlobalRegistries.makeCopyOfGlobalState(); if (injector == null) { getInjector(); } stateAfterInjectorCreation.restoreGlobalState(); }')
  # predictor.save(f'./{project}_code_bert_model')

In [7]:
for project,url in a.items():
  print(project)
  code_bert(project)

jifa
(0, 11)
preprocessing train...
language: en
train sequence lengths:
	mean : 23
	95percentile : 71
	99percentile : 128


Is Multi-Label? True
preprocessing test...
language: en
test sequence lengths:
	mean : 28
	95percentile : 102
	99percentile : 185




begin training using onecycle policy with max lr of 5e-05...
25/25 [==============================] - 396s 16s/step
              precision    recall  f1-score   support

      not_LA       0.73      0.29      0.41        28
          is       1.00      0.95      0.98        43
         set       0.51      0.89      0.65       209
         get       0.57      0.79      0.66       178

   micro avg       0.57      0.82      0.67       458
   macro avg       0.70      0.73      0.67       458
weighted avg       0.59      0.82      0.67       458
 samples avg       0.41      0.44      0.42       458

Validation Metrics: None


/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/core.py:226: UserWarning: Confusion matrices do not currently support multilabel classification, so returning None
  warnings.warn(
/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/ktrain/text/preprocessor.py:392: UserWarning: The method preproce

[[0.01789646 0.00548391 0.13518044 0.73962927]
 [0.01482094 0.00551641 0.20362128 0.7438427 ]
 [0.02315211 0.00874077 0.06019509 0.60072523]
 ...
 [0.0911471  0.9157999  0.09440434 0.0593598 ]
 [0.10412137 0.01030451 0.04564784 0.05123553]
 [0.07274462 0.00737863 0.03165642 0.23831558]]
[('not_LA', 0.048983395), ('is', 0.009222458), ('set', 0.10328331), ('get', 0.017019419)]


ValueError: probabilities do not sum to 1